In [1]:
import pandas as pd
import os
import zipfile

In [2]:
from config.config_loader import Config

config = Config()

In [3]:
db_elec_path = config.get_resolved_path("data_path.elec.processed.facturas")
db_elec_manual_path = config.get_resolved_path("data_path.elec.processed.facturas_manual")

path_alias_elec = config.get_resolved_path("data_path.elec.customers.alias_elec")

path_raw_data_elec = config.get_resolved_path("data_path.elec.invoice_comp.data_raw")
path_clientes = config.get_resolved_path("data_path.elec.customers.folder")

path_processed_files = config.get_path("data_path.elec.invoice_comp.processed_files")
path_clientes

path_recorded_cepsa = config.get_path("data_path.elec.invoice_comp.record_cepsa")
path_recorded_total = config.get_path("data_path.elec.invoice_comp.record_total")

In [4]:
import paramiko
import os
import datetime
import json

def download_sftp_files(host, port, username, password, remote_dir, local_dir, record_path):
    os.makedirs(local_dir, exist_ok=True)

    # Load previous record of downloaded files and timestamps
    if os.path.exists(record_path):
        with open(record_path, 'r') as f:
            downloaded_files = json.load(f)
    else:
        downloaded_files = {}

    # Connect to SFTP
    transport = paramiko.Transport((host, port))
    transport.connect(username=username, password=password)
    sftp = paramiko.SFTPClient.from_transport(transport)

    try:
        print(f"Conectando a {host}:{port} como {username}...")
        for attr in sftp.listdir_attr(remote_dir):
            file_name = attr.filename
            remote_path = f"{remote_dir}/{file_name}"
            local_path = os.path.join(local_dir, file_name)
            mod_time = attr.st_mtime

            # Check if file is new or modified
            prev_mod_time = downloaded_files.get(file_name)
            if prev_mod_time is None or mod_time > prev_mod_time:
                print(f"Descargando '{file_name}'...")
                sftp.get(remote_path, local_path)
                downloaded_files[file_name] = mod_time
                print(f"Guardado en: {local_path}")
            # else:
            #     print(f"'{file_name}' no ha cambiado. Se omite.")

    finally:
        sftp.close()
        transport.close()
        print(f"Conexión cerrada para {host}:{port}.")

    # Save updated record
    with open(record_path, 'w') as f:
        json.dump(downloaded_files, f, indent=2)


In [5]:
# === Parameters Total ===
host_total = "files.itcs.es"
port_total = 10022
username_total = "total"
password_total = "45c6906a-b12e-4812-b106-5a97eaa0ce2b"
remote_dir_total = "CARGA"

# === Parameters Cepsa ===
host_cepsa = "mft.cepsacorp.com"
port_cepsa = 8022
username_cepsa = "ext_CCceltica"
password_b64_cepsa = "2sRarBSAPuVi2896"
remote_dir_cepsa = "CEPSA2CELTICA"

# === Execute function ===
download_sftp_files(host_total, port_total, username_total, password_total, remote_dir_total, path_raw_data_elec, path_recorded_total)
download_sftp_files(host_cepsa, port_cepsa, username_cepsa, password_b64_cepsa, remote_dir_cepsa, path_raw_data_elec, path_recorded_cepsa)

Conectando a files.itcs.es:10022 como total...
Descargando 'ESFCLICELTICAA08449506FELEC_2500443222_20250702_113826.xml'...
Guardado en: c:\Users\mikel.perez\0_Python_projects_v2\data\invoice_comp\elec\raw_data_elec\ESFCLICELTICAA08449506FELEC_2500443222_20250702_113826.xml
Descargando 'ESFCLICELTICAA08449506FGAS_2500043212_20250708_114329.pdf'...
Guardado en: c:\Users\mikel.perez\0_Python_projects_v2\data\invoice_comp\elec\raw_data_elec\ESFCLICELTICAA08449506FGAS_2500043212_20250708_114329.pdf
Descargando 'ESFCLICELTICAB78909843FELEC_2500443294_20250702_113825.xml'...
Guardado en: c:\Users\mikel.perez\0_Python_projects_v2\data\invoice_comp\elec\raw_data_elec\ESFCLICELTICAB78909843FELEC_2500443294_20250702_113825.xml
Descargando 'ESFCLICELTICAB28159192FGAS_2500041985_20250702_113824.xml'...
Guardado en: c:\Users\mikel.perez\0_Python_projects_v2\data\invoice_comp\elec\raw_data_elec\ESFCLICELTICAB28159192FGAS_2500041985_20250702_113824.xml
Descargando 'ESFCLICELTICAB28159192FGAS_250004198

In [6]:
from invoice_comp.check_utility import extract_cups, extract_month_year, extract_num_fra_elec
import utils.utils_var as utils_var

In [7]:
df_alias_elec = pd.read_excel(path_alias_elec, sheet_name='Hoja1')
df_alias_elec.tail()

,cliente,alias,cups,tarifa,electrointensivo,contrato
105,Catalonia Fundacio,EDDA DE CARLI,ES0031408015307001CW,2.0 TD,False,1
106,Catalonia Fundacio,BESORA,ES0031405265052002WH,2.0 TD,False,1
107,Catalonia Fundacio,S JORDI 2,ES0031405060747021FN,2.0 TD,False,1
108,Catalonia Fundacio,HOSPITALET,ES0031406166448011FZ,2.0 TD,False,1
109,Generic,TRIAL,TRIAL,6.1 TD,False,1


In [8]:
output_path = os.path.abspath('../../data/invoice_comp/elec/xml_folder_clean')
raw_path = os.path.abspath('../../data/invoice_comp/elec/raw_data_elec')
fras_folder = os.path.abspath('../../data/invoice_comp/elec/facturas')
path_fras_elec_xml_clean = config.get_resolved_path("data_path.elec.invoice_comp.xml_folder_clean")
output_path

'c:\\Users\\mikel.perez\\0_Python_projects_v2\\data\\invoice_comp\\elec\\xml_folder_clean'

In [9]:
fras_list = os.listdir(path_fras_elec_xml_clean)

In [10]:
try:
    df_processed = pd.read_pickle(path_processed_files)
except:
    df_processed = pd.DataFrame(columns=['file_name'])
    
df_processed.head()

,file_name
0,029B689E04E01EEE93AA807352EF4CC7.zip
1,029B689E04E01EEE94F1E7B3F9B60CC7.zip
2,029B689E04E01EEE95D648DC01034CC7.zip
3,029B689E04E01EEE9B979DB3DC32CCC7.zip
4,029B689E04E01EEE9B9D4C2290698CC7.zip


In [11]:
raw_list = os.listdir(raw_path)
zip_list = []

for archive in raw_list:

    if '.zip' in archive or '.ZIP' in archive:
        zip_list.append(archive)
    if archive not in df_processed['file_name'].values:
        # print('total')
        if '.xml' in archive or '.XML' in archive:
            num_fra = extract_num_fra_elec(os.path.join(raw_path,archive))
            if (not num_fra + '.xml' in fras_list) or (not num_fra + '.XML' in fras_list):
                cups = extract_cups(os.path.join(raw_path,archive))
                cliente = utils_var.insert_alias(cups, df_alias_elec)[0]

                if cliente!=None:

                    month, year = extract_month_year(os.path.join(raw_path,archive))
                    month = utils_var.map_month_folder(month)
                    print(num_fra)
                    print(cliente)
                    # introducir codigo para ordenar los pdfs
                    with open(os.path.join(raw_path, archive), 'r', encoding='utf-8') as input_file:
                        xml_content = input_file.read()  # Read the content of the XML file

                        # Write the XML content to the output path
                        with open(f'{output_path}/{num_fra}{archive[-4:]}', 'w', encoding='utf-8') as output_file:
                            output_file.write(xml_content)
                            print(f"File saved to: {output_path}/{num_fra}{archive[-4:]}")

                    if f'{archive[:-4]}.pdf' in raw_list or f'{archive[:-4]}.PDF' in raw_list:

                        if not os.path.exists(os.path.abspath(f'{fras_folder}/{cliente}/{year}/{month}')):
                            os.makedirs(os.path.abspath(f'{fras_folder}/{cliente}/{year}/{month}'))

                        with open(os.path.join(f'{raw_path}', f'{archive[:-4]}.pdf'), 'rb') as input_file:
                            pdf_content = input_file.read()

                        with open(f'{fras_folder}/{cliente}/{year}/{month}/{num_fra}.pdf', 'wb') as output_file:
                            output_file.write(pdf_content)
                            print(f"File saved to: {fras_folder}/{cliente}/{num_fra}.pdf")
                            
        df_processed = pd.concat([df_processed, pd.DataFrame({'file_name': [archive]})], ignore_index=True)
        df_processed.to_pickle(path_processed_files)

for zip_file in zip_list:

    # if zip_file not in df_processed['file_name'].values:

        print('HI')

        with zipfile.ZipFile(f'{raw_path}/{zip_file}', 'r') as zip_ref:
            # List all the files inside the zip
            zip_contents = zip_ref.namelist()
            for i in zip_contents:
                if '.xml' in i or '.XML' in i:

                    with zip_ref.open(i) as file:
                        # Read the contents of the file
                        cups = extract_cups(file)

                    cliente = utils_var.insert_alias(cups, df_alias_elec)[0]

                    if cliente != None:

                        with zip_ref.open(i) as file:
                            # Read the contents of the file
                            num_fra = extract_num_fra_elec(file)
                            print(cups)
                        if (not num_fra + '.xml' in fras_list) or (not num_fra + '.XML' in fras_list):
                            print("Files in zip:", zip_contents)
                            with zip_ref.open(i) as file:
                                month, year = extract_month_year(file)
                                month = utils_var.map_month_folder(month)
                            print({num_fra})

                            with zip_ref.open(i) as file:
                                xml_content = file.read().decode('utf-8')
                            with open(f'{output_path}/{num_fra}{i[-4:]}', 'w', encoding='utf-8') as output_file:
                                output_file.write(xml_content)
                            
                            for j in zip_contents:
                                if '.pdf' in j:
                                    with zip_ref.open(j) as file:
                                        pdf_content = file.read()  # Read the PDF content in binary mode
                                        # Make sure to use 'wb' mode to write binary content
                                        if not os.path.exists(os.path.abspath(f'{fras_folder}/{cliente}/{year}/{month}')):
                                            os.makedirs(os.path.abspath(f'{fras_folder}/{cliente}/{year}/{month}'))
                                        with open(f'{fras_folder}/{cliente}/{year}/{month}/{num_fra}{j[-4:]}', 'wb') as output_file:
                                            output_file.write(pdf_content)

TOTALENERGIES ELECTRICIDAD Y G
FELEC_2500443222
Grupo Inspired
File saved to: c:\Users\mikel.perez\0_Python_projects_v2\data\invoice_comp\elec\xml_folder_clean/FELEC_2500443222.xml
File saved to: c:\Users\mikel.perez\0_Python_projects_v2\data\invoice_comp\elec\facturas/Grupo Inspired/FELEC_2500443222.pdf
TOTALENERGIES ELECTRICIDAD Y G
No alias found for cups: "ES0217901000006896WE"
TOTALENERGIES ELECTRICIDAD Y G
No alias found for cups: "ES0234150009507836RC"
TOTALENERGIES ELECTRICIDAD Y G
No alias found for cups: "ES0234150009507836RC"
TOTALENERGIES ELECTRICIDAD Y G
FELEC_2500442262
Grupo Inspired
File saved to: c:\Users\mikel.perez\0_Python_projects_v2\data\invoice_comp\elec\xml_folder_clean/FELEC_2500442262.xml
File saved to: c:\Users\mikel.perez\0_Python_projects_v2\data\invoice_comp\elec\facturas/Grupo Inspired/FELEC_2500442262.pdf
TOTALENERGIES ELECTRICIDAD Y G
FELEC_2500443293
Grupo Inspired
File saved to: c:\Users\mikel.perez\0_Python_projects_v2\data\invoice_comp\elec\xml_fold

In [12]:
fras_folder

'c:\\Users\\mikel.perez\\0_Python_projects_v2\\data\\invoice_comp\\elec\\facturas'